In [108]:
import mnist
from tqdm import tqdm

import numpy as np
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
px.imshow(x_train[0], width=500)

In [35]:
# Load the data
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

train_images = train_images.reshape((-1, 784)) / 255.0
test_images = test_images.reshape((-1, 784)) / 255.0
train_labels = np.eye(10)[train_labels]
test_labels = np.eye(10)[test_labels]

In [4]:
# convert x values from 0-255 to 0-1
x_train = x_train/255
x_test = x_test/255

# convert y values from integer to binary array
def y_categorical(y):
    res = []
    for i in range(len(y)):
        arr = [0]*10
        arr[y[i]] = 1
        res.append(arr)
    return res
y_train = y_categorical(y_train)
y_test = y_categorical(y_test)

# convert numpy to pandas
x_train = pd.DataFrame(x_train.reshape(len(x_train), 784))
x_test = pd.DataFrame(x_test.reshape(len(x_test), 784))
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)


In [211]:
class VisNet:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.w1 = np.random.randn(self.input_size, self.hidden_size)
        self.b1 = np.zeros(self.hidden_size)
        self.w2 = np.random.randn(self.hidden_size, self.output_size)
        self.b2 = np.zeros(self.output_size)

    def cross_entropy_loss(self, y_true, y_pred):
        return -np.mean(y_true * np.log(y_pred + 1e-10))

    def softmax(self,x):
        return np.exp(x)/sum(np.exp(x))
        
    def forward(self, x):
        z1 = np.dot(x, self.w1) + self.b1
        a1 = np.tanh(z1)
        z2 = np.dot(a1, self.w2) + self.b2
        y = self.softmax(z2)
        return y, a1
    
    def backward(self, x, y_true, y_pred, a1):
        delta2 = y_pred - y_true
        grad_w2 = np.dot(a1.T, delta2)
        grad_b2 = np.sum(delta2, axis=0)
        delta1 = np.dot(delta2, self.w2.T) * (1 - np.power(a1, 2))
        grad_w1 = np.dot(x.T, delta1)
        grad_b1 = np.sum(delta1, axis=0)
        return grad_w1, grad_b1, grad_w2, grad_b2
    
    def train(self, x_train, y_train, learning_rate=0.1, batch_size=128, num_epochs=10):
        num_batches = len(x_train) // batch_size
        
        for epoch in range(num_epochs):
            for batch in range(num_batches):
                # Get the batch of images and labels
                batch_images = x_train[batch*batch_size:(batch+1)*batch_size]
                batch_labels = y_train[batch*batch_size:(batch+1)*batch_size]
                
                # Forward pass
                eps = 1e-8
                y_pred, a1 = self.forward(batch_images)
                loss = self.cross_entropy_loss(batch_labels, (y_pred+eps))
                
                # Backward pass
                grad_w1, grad_b1, grad_w2, grad_b2 = self.backward(batch_images, batch_labels, y_pred, a1)
                
                # Update weights and biases
                self.w1 -= learning_rate * grad_w1
                self.b1 -= learning_rate * grad_b1
                self.w2 -= learning_rate * grad_w2
                self.b2 -= learning_rate * grad_b2
                
            # Print progress
            print("Epoch "+str(epoch+1)+"/"+str(num_epochs)+" - Loss = "+str(loss.mean()))
                    
    def evaluate(self, x_test, y_test):
        total = 0
        correct = 0
        for elem in tqdm(x_test):
            y_pred = self.forward(elem)[0].argmax()
            if(y_pred == y_test[total].argmax()):
                correct+=1
            total+=1
        # Compute accuracy
        print("Test accuracy: "+str(correct/total))


In [214]:
input_size = 784
hidden_size = 512
output_size = 10

model = VisNet(input_size, hidden_size, output_size)
model.train(train_images, train_labels, learning_rate=0.00001, num_epochs=15)

Epoch 1/15 - Loss = 1.6962783209316676
Epoch 2/15 - Loss = 1.606824640030739
Epoch 3/15 - Loss = 1.5857338208057332
Epoch 4/15 - Loss = 1.5564965350160878
Epoch 5/15 - Loss = 1.5390558967894556
Epoch 6/15 - Loss = 1.4905996262818786
Epoch 7/15 - Loss = 1.488729727422556
Epoch 8/15 - Loss = 1.5061870106995048
Epoch 9/15 - Loss = 1.5248004315478316
Epoch 10/15 - Loss = 1.5380892074020365
Epoch 11/15 - Loss = 1.5395911056112117
Epoch 12/15 - Loss = 1.5310547917504789
Epoch 13/15 - Loss = 1.5154530066272929
Epoch 14/15 - Loss = 1.5090920050110757
Epoch 15/15 - Loss = 1.4965211161523269


In [215]:
model.evaluate(test_images,test_labels)

100%|██████████| 10000/10000 [00:01<00:00, 6128.52it/s]

Test accuracy: 0.6172
